In [10]:
from langchain.agents import Tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage
import os
import json
import getpass
import tempfile
import subprocess
import re

# Set up Gemini API key
if not os.environ.get("GOOGLE_API_KEY"):
    try:
        from google.colab import userdata
        os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')
    except:
        import getpass
        os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter Google API Key: ")

class PlannerAgent:
    def __init__(self):
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest",
            temperature=0.3,
            convert_system_message_to_human=True
        )
        self.system_prompt = """
        As an expert solution architect, decompose complex problems into executable sub-tasks.
        Use this JSON structure:
        {
            "subtasks": [
                {
                    "id": <unique integer>,
                    "desc": "<clear description>",
                    "dependencies": [<list of prerequisite task IDs>]
                }
            ]
        }
        """

    def decompose(self, task):
        try:
            messages = [
                SystemMessage(content=self.system_prompt),
                HumanMessage(content=f"TASK: {task}")
            ]
            llm_response = self.llm.invoke(messages)
            response_content = llm_response.content
            if '```json' in response_content:
                json_match = re.search(r'```json(.*?)```', response_content, re.DOTALL)
                if json_match:
                    response_content = json_match.group(1).strip()
            plan = json.loads(response_content)
            if "subtasks" not in plan or not isinstance(plan["subtasks"], list):
                raise ValueError("Invalid response format: Missing 'subtasks' list")
            for subtask in plan["subtasks"]:
                if "id" not in subtask or "desc" not in subtask:
                    raise ValueError("Subtask missing required fields: 'id' or 'desc'")
            return plan
        except Exception as e:
            print(f"Decomposition error: {str(e)}")
            return self._fallback_plan(task)

    def _fallback_plan(self, task):
        return {
            "subtasks": [
                {"id": 1, "desc": f"Analyze requirements: {task}", "dependencies": []},
                {"id": 2, "desc": f"Design solution architecture for {task}", "dependencies": [1]},
                {"id": 3, "desc": f"Implement core functionality for {task}", "dependencies": [2]},
                {"id": 4, "desc": f"Test and validate solution for {task}", "dependencies": [3]}
            ]
        }

class ExecutorAgent:
    def __init__(self):
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest",
            temperature=0.2,
            convert_system_message_to_human=True
        )

    def write_code(self, requirements: str) -> dict:
        prompt = (
            f"You are an expert Python developer. "
            f"Write complete, self-contained Python code to {requirements}. "
            f"Return only the code block, wrapped in triple backticks for Python."
        )
        llm_response = self.llm.invoke(prompt).content
        match = re.search(r"```(?:python)?\n([\s\S]*?)```", llm_response)
        code = match.group(1) if match else llm_response
        tmp_dir = tempfile.mkdtemp()
        file_path = os.path.join(tmp_dir, "main.py")
        with open(file_path, "w") as f:
            f.write(code)
        return {"code": code, "files": [file_path]}

    def run_tests(self, code_info: dict) -> dict:
        files = code_info.get("files", [])
        if not files:
            return {"passed": False, "errors": ["No file to execute."]}
        file_to_run = files[0]
        try:
            result = subprocess.run(
                ["python", file_to_run],
                capture_output=True, text=True, timeout=10
            )
            if result.returncode == 0:
                return {"passed": True, "errors": []}
            else:
                return {"passed": False, "errors": [result.stderr.strip()]}
        except Exception as e:
            return {"passed": False, "errors": [str(e)]}

class ReviewerAgent:
    def __init__(self):
        self.llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest",
            temperature=0.2,
            convert_system_message_to_human=True
        )

    def analyze(self, code_result: dict, test_result: dict) -> dict:
        code = code_result.get("code", "")
        tests = test_result.get("errors", []) or []
        tests_output = "\n".join(tests) if tests else "All tests passed."
        prompt = (
            "You are a senior software engineer and code reviewer. "
            "Review the following Python code snippet and its test results. "
            "Provide a JSON object with two keys:\n"
            " - feedback: A concise summary of overall code quality.\n"
            " - critical_issues: A list of strings describing any bugs, logical errors, "
            "or style issues that should be fixed.\n\n"
            f"Code:\n```python\n{code}\n```\n\n"
            f"Test Results:\n{tests_output}\n\n"
            "Output JSON only."
        )
        llm_response = self.llm.invoke(prompt).content
        try:
            json_str = re.search(r"\{[\s\S]*\}", llm_response).group(0)
            result = json.loads(json_str)
            feedback = result.get("feedback", "")
            critical_issues = result.get("critical_issues", [])
        except Exception:
            feedback = llm_response.strip()
            critical_issues = []
        return {"feedback": feedback, "critical_issues": critical_issues}

class Orchestrator:
    def __init__(self):
        self.planner = PlannerAgent()
        self.executor = ExecutorAgent()
        self.reviewer = ReviewerAgent()
        self._generated = []
        self.integrator_llm = ChatGoogleGenerativeAI(
            model="gemini-1.5-flash-latest",
            temperature=0.1,
            convert_system_message_to_human=True
        )

    def integrate_code(self, code_snippets: list) -> str:
        if not code_snippets:
            return "# No code snippets to integrate."
        combined_snippets = "\n\n# --- SNIPPET ---\n\n".join(code_snippets)
        integration_prompt = (
            "You are an expert software integrator. You have been given several Python code snippets. "
            "Your task is to combine them into a single, cohesive, and functional Python script.\n\n"
            "Perform the following actions:\n"
            "1. Merge all imports at the top and remove duplicates.\n"
            "2. Logically combine classes and functions. If functions with the same name exist, merge them intelligently.\n"
            "3. Remove redundant code, placeholder comments, and example usage blocks.\n"
            "4. Ensure the final script is clean, correct, and executable.\n"
            "5. Return ONLY the final, integrated Python code, wrapped in a single triple-backtick block.\n\n"
            f"Here are the code snippets to integrate:\n\n{combined_snippets}"
        )
        messages = [
            SystemMessage(content="You are an expert Python code integrator."),
            HumanMessage(content=integration_prompt)
        ]
        llm_response = self.integrator_llm.invoke(messages).content
        match = re.search(r"```(?:python)?\n([\s\S]*?)```", llm_response)
        integrated_code = match.group(1).strip() if match else llm_response.strip()
        return integrated_code

    def execute_project(self, user_request):
        plan = self.planner.decompose(user_request)
        if not plan or "subtasks" not in plan or not isinstance(plan["subtasks"], list):
            print("Planner returned an invalid plan.")
            return "Project execution failed: Invalid plan."
        self._generated = []
        for subtask in plan["subtasks"]:
            if not isinstance(subtask, dict) or "desc" not in subtask:
                print(f"Skipping invalid subtask: {subtask}")
                continue
            print(f"\n🔧 Processing subtask {subtask['id']}: {subtask['desc']}")
            code_result = self.executor.write_code(subtask["desc"])
            if not code_result or "code" not in code_result:
                print(f"Executor failed to generate code for subtask: {subtask['desc']}")
                continue
            self._generated.append(code_result["code"])
            print("🧪 Running tests...")
            test_result = self.executor.run_tests(code_result)
            if not test_result or "passed" not in test_result:
                print(f"Executor failed to run tests for subtask: {subtask['desc']}")
                continue
            if not test_result["passed"]:
                print("🔍 Reviewing failed tests...")
                review = self.reviewer.analyze(code_result, test_result)
                print(f"📝 Review feedback: {review.get('feedback', 'No feedback')}")
                if review.get("critical_issues"):
                    print(f"❌ Critical issues: {review['critical_issues']}")
            else:
                print("✅ Tests passed")
        print("\n\n⚙️ Integrating all generated code snippets into a final script...")
        if not self._generated:
            return "No code generated for any subtasks"
        final_code = self.integrate_code(self._generated)
        print("✅ Integration complete.")
        return final_code

# Example Usage:
if __name__ == "__main__":
    system = Orchestrator()
    request = "Create a Flask app with a single endpoint that returns 'Hello, World!'."
    final_script = system.execute_project(request)
    print("\n--- FINAL INTEGRATED SCRIPT ---")
    print(final_script)


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



🔧 Processing subtask 1: Create a new Python virtual environment.


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


🧪 Running tests...
✅ Tests passed

🔧 Processing subtask 2: Install Flask within the virtual environment: `pip install Flask`


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


...

🧪 Running tests...
🔍 Reviewing failed tests...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


📝 Review feedback: The code is mostly well-structured and functional, but has a critical security vulnerability and a minor usability issue.
❌ Critical issues: ['The application uses `app.run(debug=True)`.  Debug mode should **never** be enabled in a production environment.  It exposes the application to significant security risks.', 'The `app.secret_key` is generated using `os.urandom(24)`, which is insufficient for production.  A cryptographically secure random number generator should be used, and the key should be stored securely (e.g., environment variable).', 'Error handling for file uploads could be improved.  The current implementation only provides a text message for invalid file types; more informative feedback to the user would enhance usability.', 'The application lacks input validation beyond checking file extensions.  This leaves it vulnerable to various attacks (e.g., directory traversal). More robust validation is needed.']


⚙️ Integrating all generated code snippets in

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:483: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


✅ Integration complete.

--- FINAL INTEGRATED SCRIPT ---
import json
import os
import subprocess
import sys
from flask import Flask, jsonify, request, render_template, redirect, url_for
from urllib.parse import urljoin
import uuid
from werkzeug.utils import secure_filename
import venv
import requests
import pytest


def create_virtualenv(venv_path, python_executable=None):
    if not os.path.isdir(venv_path):
        raise ValueError(f"Invalid venv_path: '{venv_path}' is not a directory.")

    if python_executable:
        if not os.path.exists(python_executable):
            raise FileNotFoundError(f"Python executable not found: {python_executable}")
        python_exe = python_executable
    else:
        python_exe = sys.executable

    try:
        subprocess.run([python_exe, "-m", "venv", venv_path], check=True, capture_output=True, text=True)
        print(f"Virtual environment created successfully at: {venv_path}")
        return venv_path
    except subprocess.CalledProcessErr

In [2]:
!pip install langchain-google-genai flask langchain langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 929.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
